# QUEUES

## NOTE: this needs to be run in docker because graphviz is a pain to install in OS X


## 004. THIS IS ABOUT...

https://realpython.com/queue-in-python/

Queues are the backbone of numerous algorithms found in games, artificial intelligence, satellite navigation, and task scheduling. They’re among the top abstract data types that computer science students learn early in their education. At the same time, software engineers often leverage higher-level message queues to achieve better scalability of a microservice architecture. Plus, using queues in Python is simply fun!

Python provides a few built-in flavors of queues that you’ll see in action in this tutorial. You’re also going to get a quick primer on the theory of queues and their types. Finally, you’ll take a look at some external libraries for connecting to popular message brokers available on major cloud platform providers.

In this tutorial, you’ll learn how to:

    Differentiate between various types of queues
    Implement the queue data type in Python
    Solve practical problems by applying the right queue
    Use Python’s thread-safe, asynchronous, and interprocess queues
    Integrate Python with distributed message queue brokers through libraries


In [1]:
%load_ext nb_mypy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # type: ignore

Version 1.0.5


In [2]:
from collections import deque
from heapq import heappop, heappush
import networkx as nx
from typing import Any, Callable, Deque, Generic, NamedTuple, TypeVar
from enum import IntEnum

In [3]:
roadmap_file = "004_roadmap.dot"


### 001.001 Building a Queue Data Type

Because you want your custom FIFO queue to support at least the enqueue and dequeue operations, go ahead and write a bare-bones Queue class that’ll delegate those two operations to `deque`


1. Create a BasicQueue FIFO queue class which
    1. starts with an empty queue - make sure the elements are stored in `_elements` and are all of the same type, and the class knows about it
    1. has an `enqueue` method to add items to the queue
    1. has a `deque` method which gets it off it
    1. uncomment the assertions to prove they work
1. Create a slightly more complex Queue class which
    1. can take a list of elements
    1. enqueue and dequeue as before
    1. allows `len(an_instance)` to return the length
    1. allows `for i in an_instance...` to iterate through items
    1. uncomment the assertions to prove they work


In [4]:


T = TypeVar('T')

1
# class BasicQueue...something T...:
#     def __init__(self):
#         """1.1"""

#     def enqueue(self, element: T) -> None:
#         """1.2"""

#     def dequeue(self) -> T:
#         """1.3"""

# 1.4
# basic_fifo = BasicQueue()
# basic_fifo.enqueue(1)
# basic_fifo.enqueue(2)
# basic_fifo.enqueue(3)


# assert basic_fifo.dequeue() == 1
# assert basic_fifo.dequeue() == 2
# assert basic_fifo.dequeue() == 3

2
# class Queue...something T...:
#     def __init__(self, extend_to, any, number):
#         """2.1"""

#     def enqueue(self, element: T) -> None:
#         """2.2"""

#     # def dequeue(self) -> T:
#     #     """2.2"""

#     2.3
#     """length"""

#     2.4
#     """for..."""

2.5
# fifo = Queue(1, 2, 3)
# assert len(fifo) == 3
# assert [1,2,3] == [el for el in fifo]
# assert len(fifo) == 0

# fifo.enqueue(1)
# fifo.enqueue(2)
# fifo.enqueue(3)

# assert fifo.dequeue() == 1
# assert fifo.dequeue() == 2
# assert fifo.dequeue() == 3

# solution


<cell>63: note: By default the bodies of untyped functions are not checked, consider using --check-untyped-defs  [annotation-unchecked]


1

2

2.5

2.5

### 001.002 Building a Stack Data Type

Building a stack data type is considerably more straightforward because you’ve already done the bulk of the hard work. Since most of the implementation will remain the same, you can extend your Queue class using inheritance and override the .dequeue() method to remove elements from the top of the stack

(NOTE: this doesn't imply that conceptually Stack extends Queue, it's simply for convenience)

1. Create a Stack class
    1. It inherits from Queue
    1. It overwrites the dequeue method to remove elements from the top
    1. uncomment assertions

In [5]:
1
# class Stack(Queue ...something T...):
#     def dequeue(self):
#         """1.1"""

# lifo = Stack(1, 2, 3)
# assert len(lifo) == 3
# assert [3,2,1] == [el for el in lifo]
# assert len(lifo) == 0

# solution


1

### 001.003 Building a Priority Queue

Fortunately, you can be smart about keeping the elements sorted in a priority queue by using a heap data structure under the hood. It provides the most efficient implementation

Python has the heapq module, which conveniently provides a few functions that can turn a regular list into a heap and manipulate it efficiently.

1. Create a PriorityQueue class
    1. Initialised with 
        1. It stores elements in a simple list
        1. It uses a standard generator from itertools to keep track of next item
        1. ...which needs to be imported
    1. enqueue_with_priority leverages Python’s tuple comparison, which takes into account the tuple’s components, looking from left to right until the outcome is known
        1. Elements contain the priority first...
            1. Don't forget to type annotate element
            1. Q Note that heapq is a min-heap, i.e..... How does that change the way priority is encoded? 
            1. We would like to maintain the chronological ordering of queue items if they have the same priority. That's what the counter is all about
            1. Finally the value
        1. Then the element is finally pushed into the heap
    1. Dequeue only fetches the value from the helement, letting heapq do the work
        

In [6]:
class Priority(IntEnum):
    CRITICAL = 3
    IMPORTANT = 2
    NEUTRAL = 1

1
# class PriorityQueue:
#     def __init__(self):
#         """1.1.1"""
#         """1.1.2"""

#     def enqueue_with_priority(self, priority:Priority, value:str) -> None:
#         """1.2.1"""
#         element = ...
#             # 1.2.1.1 Q: Note that heapq is a min-heap, i.e....
#             # 1.2.1.2
#             # 1.2.1.3
#             # 1.2.1.4
#         # 1.2.2

#     def dequeue(self) -> str:
#         """1.3"""

# 2
# messages = PriorityQueue()
# messages.enqueue_with_priority(Priority.IMPORTANT, "Windshield wipers turned on")
# messages.enqueue_with_priority(Priority.NEUTRAL, "Radio station tuned in")
# messages.enqueue_with_priority(Priority.CRITICAL, "Brake pedal depressed")
# messages.enqueue_with_priority(Priority.IMPORTANT, "Hazard lights turned on")

# assert messages.dequeue() == 'Brake pedal depressed'

# solution


<cell>40: note: By default the bodies of untyped functions are not checked, consider using --check-untyped-defs  [annotation-unchecked]


1

1

2

### 001.004 Refactoring with mixins

`len` and `__iter__` are common to all 3 classes so far, and can be espressed as mixins. Assume `_elements` is common to all

1. Extract `__len__` and `__iter__` from earlier Queue class
1. Extend all classes created so far (Queue, Stack, PriorityQueue) with it



In [7]:
1
class IterableMixin:
    def __len__(self) -> int:
        return len(self._elements) # type: ignore

    def __iter__(self):
        while len(self) > 0:
            yield self.dequeue()

# class Queue(IterableMixin):
#     ...

# class Stack(Queue):
#     ...

# class PriorityQueue(IterableMixin):
#     ...

# fifo = Queue(1, 2, 3)
# assert len(fifo) == 3
# assert [1,2,3] == [el for el in fifo]
# assert len(fifo) == 0

# lifo = Stack(1, 2, 3)
# assert len(lifo) == 3
# assert [3,2,1] == [el for el in lifo]
# assert len(fifo) == 0

# messages = PriorityQueue()
# messages.enqueue_with_priority(Priority.IMPORTANT, "Windshield wipers turned on")
# messages.enqueue_with_priority(Priority.NEUTRAL, "Radio station tuned in")
# messages.enqueue_with_priority(Priority.CRITICAL, "Brake pedal depressed")
# messages.enqueue_with_priority(Priority.IMPORTANT, "Hazard lights turned on")

# assert messages.dequeue() == 'Brake pedal depressed'

# solution


<cell>55: note: By default the bodies of untyped functions are not checked, consider using --check-untyped-defs  [annotation-unchecked]


1

### 001.005 Exploring networkx, graphviz and DOT

The DOT graph description language defines a file format that is most often used in the context of graph visualization with Graphviz. NetworkX provides an interface to Graphviz via pygraphviz, implemented in nx_agraph. If pygraphviz is installed, nx_agraph can be used to read and write files in DOT format.


1. Read `roadmap_file` into var `graph` as a networkx data structure, using the `pygraphviz` extension
    1. networkx graph nodes are textual identifiers that can optionally have an associated dictionary of attributes. Print the one for "london" to see what it looks like
2. Instead of a dict we want a data structure which is hashable but less["awkward"] to work with. Create a class City
    1. the `from_dict` method takes the dict from 1.1 and returns a City instance
    1. note that year could be not found
    1. make sure you cast all non-string types
3. Create a couple of helpers function
    1. `as_cities_map` should cycle through `graph.nodes` (from 1.) and create a similar dict but with City (from 2.) as value
        1. TIP: you need to pass `data=True` when iterating though nodes, so that you get a dict
    1. create `cities_map` with it
    1. `as_cities_graph` should create a `nx.Graph` by cycling through `graph.edges` (from 1.) and for each edge passing the tuple `name1, name2, weights` as arg to it
        1. and replacing `nameX` with a City from 3.2
        1. TIP: you need to pass `data=True` when iterating though nodes, so that you get a dict
    1. create a var `cities_graph` with it
4. Explore by creating a couple of helpers and using them to get a list of neighbours ordered by distance
    1. cities_graph[cities_map["london"]] returns a dictionary, where the key is a `City()` instance, and the value a dictionary in form `{'distance': '53', 'label': '53'})`. They are ordered by insertion order
    1. Create a function `sort_by` which takes each item in the dictionary and sorts it by passing the value to a callable (passed as one of the argument)
    1. Create another function, `by_distance`, which is the callable we shall call this time
        1. Cast the return value to be sure
    1. Should give a list of neightbours arranaged by distance



In [8]:


# class City(NamedTuple):
#     name: str
#     country: str
#     year: int | None
#     latitude: float
#     longitude: float

#     @classmethod
#     def from_dict(cls, attrs:dict[str, Any]) -> "City":
#         2

3.1
# def as_cities_map(
#         agraph: nx.Graph,
#         node_factory: Callable[[dict[str, Any]], Any]
#     ) -> dict[str, Any]:
#     ...

3.3
# def as_cities_graph(cities_map:dict[str, City], agraph: nx.Graph) -> nx.Graph:
#     ...
4.2
# def sort_by(neighbors: dict[str, City], strategy: Callable[[dict[str, Any]], Any]):
#     return sorted(..., key=lambda item: ...)

4.3
# def by_distance(weights: dict[str, Any]):
#     ...
4.4
# for neighbor, weights in sort_by(cities_graph[cities_map["london"]], by_distance):
#     weights["distance"], neighbor.name

# solution


3.1

3.3

4.2

4.3

4.4

...

('79', 'Southampton')

('85', 'Peterborough')

('100', 'Coventry')

('115', 'Bath')

('118', 'Bristol')